In [ ]:
!pip install pomdp-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## General setup

In [ ]:
import pomdp_py
import numpy as np
import random

In [ ]:
from collections import defaultdict

def tuple_default():
    return defaultdict((float, int))

In [ ]:
import random

class Q_Learning_Table:
  def __init__(self, gamma, n, n_actions) -> None:
    self.q_tab = defaultdict(tuple_default)
    self.gamma = gamma
    self.n = n
    self.n_actions = n_actions

  def update(self, goal, o_t, a_t, r_t, new_o):
    #TODO: Ler sobre flattening policy
    a_v = self.q_tab[(goal, new_o)]
    best_a = None
    max_v = float('-inf')

    for a, (v, _) in a_v.items():
      if v > max_v:
        max_v = v
        best_a = a

    if not best_a:
      best_a = random.randint(1, self.n_actions)

    new_value = r_t + self.gamma * max_v

    if (goal, o_t) not in self.q_tab or (a_t not in self.q_tab[(goal, o_t)]):
      self.q_tab[(goal, o_t)][a_t] = new_value
    else:
      v, N = self.q_tab[(goal, o_t)][a_t]
      alpha = 1/N
      self.q_tab[(goal, o_t)][a_t] = (1 - alpha) * self.q_tab[(goal, o_t)][a_t] \
        + alpha * new_value


class ExperienceMemory:
  def __init__(self, state_value_tab : Q_Learning_Table) -> None:
    self.state_value_tab = state_value_tab
    self.experiences = []

  def add_experience(self, goal, o_t, a_t, r_t, new_o):
    self.experiences.append((goal, o_t, a_t, r_t, new_o))
    self.state_value_tab.update(goal, o_t, a_t, r_t, new_o)

  def __hash__(self):
      return hash(experiences)

  def __eq__(self, other):
    if isinstance(other, State):
      return self.experiences == other.experiences
    else:
      return False

  def __str__(self):
    return f"""Experience memory: {self.experiences}\n"""

  def __repr__(self):
    return f"""Experience memory: {self.experiences}\n"""


In [ ]:
class State(pomdp_py.State):
    def __init__(self, exp_mem : ExperienceMemory):
        self.exp_mem = exp_mem

    def __hash__(self):
      return hash(self.exp_mem)

    def __eq__(self, other):
      if isinstance(other, State):
        return self.exp_mem == other.exp_mem
      else:
        return False

    def __str__(self):
      return f"""Experience memory: {self.exp_mem}\n"""

    def __repr__(self):
      return f"""Experience memory: {self.exp_mem}\n"""

In [ ]:
class Action(pomdp_py.Action):
    """Simple named action."""
    def __init__(self, name):
        self.name = name
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        if isinstance(other, Action):
            return self.name == other.name
        elif type(other) == str:
            return self.name == other
    def __str__(self):
        return self.name
    def __repr__(self):
        return "Action(%s)" % self.name

In [ ]:
class Observation(pomdp_py.Observation):
    def __init__(self, screen: str):
        self.screen = screen

    def __hash__(self):
      return hash(self.screen)

    def __eq__(self, other):
        return self.screen == other.screen

    def __str__(self):
      return f"""Screen: {self.screen}\n"""

    def __repr__(self):
      return f"""Screen: {self.screen}\n"""

In [ ]:
class ObservationModel(pomdp_py.ObservationModel):
  def probability(self, observation, next_state, action):
    #Vamos assumir que a observação, no caso a descrição da tela, é fidedigna.  Portanro, esta não é uma fonte de incerteza.
    return 1.0

  def sample(self, next_state: State, action):
    exp = next_state.exp_mem.experiences
    return exp[len(exp) - 1][1]

In [ ]:
class TransitionModel(pomdp_py.TransitionModel):
  def probability(self, next_state, state, action):
    return 1.0 - 1e-9

  def sample(self, state, action):
    #TODO: Aqui, vamos precisar do env para executar a ação no novo estado
    pass


## WebShop Case Study

### Installation and setup (random policy)

In [ ]:
#Util files
!git clone https://github.com/monilouise/IN1087.git

Cloning into 'IN1087'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 9 (delta 0), pack-reused 0
Receiving objects: 100% (9/9), 26.89 KiB | 4.48 MiB/s, done.


In [ ]:
!git clone https://github.com/princeton-nlp/webshop.git webshop

Cloning into 'webshop'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 395 (delta 45), reused 40 (delta 40), pack-reused 338
Receiving objects: 100% (395/395), 46.69 MiB | 28.89 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [ ]:
!cp -f IN1087/webshop/requirements.txt webshop

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:19
🔁 Restarting kernel...


In [ ]:
%cd webshop
!./setup.sh -d small

/content/webshop
  Using cached numpy-1.22.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2
/usr/local/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:19: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bind

In [ ]:
!conda install mkl=2021

/usr/local/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:19: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/usr/local/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
| / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \

In [ ]:
import gym
from web_agent_site.envs import WebAgentTextEnv

env = gym.make('WebAgentTextEnv-v0', observation_mode='text', num_products=1000)

Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 37036.45it/s]


Loaded 6910 goals.


/usr/local/lib/python3.10/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(


In [ ]:
import gym
from rich import print
from rich.markup import escape

from web_agent_site.envs import WebAgentTextEnv
from web_agent_site.models import RandomPolicy
from web_agent_site.utils import DEBUG_PROD_SIZE

env = gym.make('WebAgentTextEnv-v0', observation_mode='text', num_products=DEBUG_PROD_SIZE)
env.reset()

try:
    policy = RandomPolicy()

    observation = env.observation
    while True:
        print(observation)
        available_actions = env.get_available_actions()
        print('Available actions:', available_actions)
        action = policy.forward(observation, available_actions)
        observation, reward, done, info = env.step(action)
        print(f'Taking action "{escape(action)}" -> Reward = {reward}')
        if done:
            break
finally:
    env.close()

In [ ]:
env.reset()
observation = env.observation
observation

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


"WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Search"

In [ ]:
env.get_available_actions()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'has_search_bar': True, 'clickables': ['search']}

In [ ]:
observation

"Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 1 (Total results: 0) [SEP] Next >"

In [ ]:
observation, reward, done, info = env.step('click[back to search]')

In [ ]:
observation

"WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Search"

In [ ]:
observation, reward, done, info = env.step('search[bags]')

In [ ]:
observation

"Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 1 (Total results: 0) [SEP] Next >"

In [ ]:
%cd /content/webshop

/content/webshop


In [ ]:
!pip install numpy --upgrade

  Using cached numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


In [ ]:
!./run_web_agent_text_env.sh

Streaming output truncated to the last 5000 lines.
Taking action "click[next >]" -> Reward = 0.0
Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & overalls with short 
sleeve, high waist, polyester spandex for daily wear with color: green stripe, and size: large, and 
price lower than 50.00 dollars [SEP] Back to Search [SEP] Page 2 (Total results: 0) [SEP] < Prev 
[SEP] Next >
Available actions:
{'has_search_bar': False, 'clickables': ['back to search', '< prev', 'next >']}
Taking action "click[back to search]" -> Reward = 0.0
WebShop [SEP] Instruction: [SEP] Find me slim fit, machine wash women's jumpsuits, rompers & 
overalls with short sleeve, high waist, polyester spandex for daily wear with color: green stripe, 
and size: large, and price lower than 50.00 dollars [SEP] Search
Available actions:
{'has_search_bar': True, 'clickables': ['search']}
Taking action "search[shoes]" -> Reward = 0.0
Instruction: [SEP] Find me slim fit, machine wash women's jumps

### WebShop Baseline Models

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/IN1087/Projeto/webshop/webshop_ckpts

choice_il_epoch9.pth  search_il_checkpoint_800.zip


In [ ]:
!mkdir -p ckpts/web_click/epoch_9/
!mkdir -p ckpts/web_search/

In [ ]:
!cp /content/drive/MyDrive/IN1087/Projeto/webshop/webshop_ckpts/choice_il_epoch9.pth model.pth
!mv model.pth ckpts/web_click/epoch_9/
!cp /content/drive/MyDrive/IN1087/Projeto/webshop/webshop_ckpts/search_il_checkpoint_800.zip ckpts/web_search/
!unzip ckpts/web_search/search_il_checkpoint_800.zip

Archive:  ckpts/web_search/search_il_checkpoint_800.zip
   creating: checkpoint-800/
  inflating: __MACOSX/._checkpoint-800  
  inflating: checkpoint-800/rng_state.pth  
  inflating: checkpoint-800/optimizer.pt  
  inflating: checkpoint-800/config.json  
  inflating: checkpoint-800/scheduler.pt  
  inflating: checkpoint-800/training_args.bin  
  inflating: checkpoint-800/pytorch_model.bin  
  inflating: checkpoint-800/trainer_state.json  


In [ ]:
%cd /content/webshop/baseline_models

/content/webshop/baseline_models


In [ ]:
!pip install datasets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2


In [ ]:
!python train_search_il.py

Downloading: 100% 878k/878k [00:00<00:00, 15.2MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 10.4MB/s]
Downloading: 100% 26.0/26.0 [00:00<00:00, 69.5kB/s]
Downloading: 100% 1.59k/1.59k [00:00<00:00, 1.18MB/s]
train size: 1479
validation size: 68
test size: 704
all size: 12251
{'input_ids': tensor([    0, 11802,  1836,  1104,  1437, 20585,   112,    73,   306, 23595,
         3524,  6399,   251, 24150,  7714,  2999,  2137,    19, 23637, 11692,
         1755, 14187,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
      

In [ ]:
!pip install wandb

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.1 MB/s eta 0:00:00


In [ ]:
!python test.py --model_path /content/webshop/ckpts/web_click/epoch_9/ --bart_path /content/webshop/checkpoint-800/

/usr/local/lib/python3.10/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/usr/local/lib/python3.10/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Products loaded.
Keys cleaned.
Attributes loaded.
100% 1000/1000 [00:00<00:00, 74452.90it/s]
0 skipped
Loaded 13 goals.
Traceback (most recent call last):
  File "/content/webshop/baseline_models/test.py", line 9, in <module>
    env = WebEnv(args, split='test')
  File "/content/webshop/baseline_models/env.py", line 21, in _